In [48]:
from pathlib import Path
import random
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from scipy.interpolate import griddata


In [49]:
DATA_PATH      = Path("../data/processed/nitrate_00_train_data.parquet")
TARGET         = "nitrate_00"
TEST_BBOX      = {                       # Morze Śródziemne
    "lat_min": 30.0, "lat_max": 46.0,
    "lon_min": -6.0, "lon_max": 36.0
}
SEED           = 42
N_JOBS         = -1
SUB_FRAC       = 0.20  # bo RAM sie konczy xD 100% train_df zabiera 25GB RAMu, mozna inne podejscia spróbować


In [50]:
random.seed(SEED)
np.random.seed(SEED)

df = pd.read_parquet(DATA_PATH)

In [51]:

deg2rad = np.pi / 180.0
df["sin_lat"]   = np.sin(df["lat"] * deg2rad)
df["cos_lat"]   = np.cos(df["lat"] * deg2rad)
df["sin_lon"]   = np.sin(df["lon"] * deg2rad)
df["cos_lon"]   = np.cos(df["lon"] * deg2rad)
df["norm_depth"] = df["depth"] / df["depth"].max()


In [52]:
cols = ["temperature_00", "salinity_00", "oxygen_00", "phosphate_00"]

In [53]:
print(f"Data shape: {df.shape}")
df = df.dropna(subset=cols).reset_index(drop=True)
print(f"Data shape after dropping NaNs: {df.shape}")

Data shape: (703450, 13)
Data shape after dropping NaNs: (609912, 13)


In [54]:
FEATURES = [
    "temperature_00", "oxygen_00", "salinity_00", "phosphate_00",
    "sin_lat", "cos_lat", "sin_lon", "cos_lon", "norm_depth"
]

test_mask = (
    (df["lat"].between(TEST_BBOX["lat_min"], TEST_BBOX["lat_max"])) &
    (df["lon"].between(TEST_BBOX["lon_min"], TEST_BBOX["lon_max"]))
)

train_df = df.loc[~test_mask].reset_index(drop=True)
test_df  = df.loc[test_mask].reset_index(drop=True)

train_df   = train_df.sample(frac=SUB_FRAC, random_state=SEED).reset_index(drop=True)

print(f"Train samples: {len(train_df):,d}")
print(f"Test  samples: {len(test_df):,d}  (bbox: {TEST_BBOX})")

Train samples: 120,279
Test  samples: 8,519  (bbox: {'lat_min': 30.0, 'lat_max': 46.0, 'lon_min': -6.0, 'lon_max': 36.0})


In [55]:

X_train, y_train = train_df[FEATURES].values, train_df[TARGET].values
X_test,  y_test  = test_df[FEATURES].values,  test_df[TARGET].values

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

In [56]:

def evaluate(name, y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    print(f"{name:<18} MSE={mse:.4f} RMSE={rmse:.4f} MAE={mae:.4f}  R2={r2:.4f}")

In [57]:
rf_pipe = make_pipeline(
    SimpleImputer(strategy="median"),
    RandomForestRegressor(n_estimators=400, n_jobs=-1, random_state=SEED)
)

rf_pipe.fit(X_train, y_train)
pred_rf = rf_pipe.predict(X_test)
evaluate("RandomForest", y_test, pred_rf)

RandomForest       MSE=3.2998 RMSE=1.8165 MAE=1.3604  R2=0.6618


In [58]:

xgb = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    reg_lambda=1.0,
    device='cuda',
    n_jobs=N_JOBS,
    random_state=SEED,
)
xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)
evaluate("XGBoost", y_test, pred_xgb)

/home/jadwiga/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:54:21] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


XGBoost            MSE=12.6218 RMSE=3.5527 MAE=2.6112  R2=-0.2934


In [59]:

train_pts = train_df[["lat", "lon", "depth"]].values
test_pts  = test_df[["lat", "lon", "depth"]].values
val_train = y_train

pred_lin = griddata(train_pts, val_train, test_pts, method="linear")
# fallback na najbliższy sąsiad tam, gdzie linear == NaN
nan_mask = np.isnan(pred_lin)
if nan_mask.any():
    pred_nn = griddata(train_pts, val_train, test_pts[nan_mask], method="nearest")
    pred_lin[nan_mask] = pred_nn
evaluate("Linear interp 3D", y_test, pred_lin)

Linear interp 3D   MSE=61.0012 RMSE=7.8103 MAE=4.8002  R2=-5.2512


In [60]:

out = test_df[["lat", "lon", "depth"]].copy()
out["nitrate_true"] = y_test
out["rf_pred"]       = pred_rf
out["xgb_pred"]      = pred_xgb
out["interp_pred"]   = pred_lin
out.to_csv("../results/baseline_predictions_nitrate.csv", index=False)
print(">> Wyniki zapisane do ../results/baseline_predictions_nitrate.csv")


>> Wyniki zapisane do ../results/baseline_predictions_nitrate.csv
